In [1]:
!pip3 install tensorflow tensorflow-hub numpy
import os
## to get the tensorflow model that can generate embeddings
import tensorflow as tf 
import tensorflow_hub as hub
import numpy as np
import re
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.6 MB/s eta 0:00:0000:0100:03m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 3.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 983.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.3 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 10.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 8.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.2 MB/s

2024-03-23 10:13:42.763595: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 10:13:42.803362: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 10:13:42.868381: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-23 10:13:43.881645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Download the model from tensorflow hub and load it

model_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" 
model = hub.load(model_url)

2024-03-23 10:15:10.340020: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 17067520 exceeds 10% of free system memory.
2024-03-23 10:15:10.422171: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 17067520 exceeds 10% of free system memory.
2024-03-23 10:15:10.442917: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 17067520 exceeds 10% of free system memory.
2024-03-23 10:15:10.535477: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 17067520 exceeds 10% of free system memory.
2024-03-23 10:15:10.610764: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 17067520 exceeds 10% of free system memory.


In [5]:
# function to generate embeddings

def embeddings(text):
    return np.array(model(text)).flatten().tolist()

In [7]:
# dataset downsloaded from https://archive.ics.uci.edu/dataset/228/sms+spam+collection
# create embeddings from this dataset using universal-sentence-encoder model

file_path = '../../SMSSpamCollection'

with open(file_path) as file:
    lines = [line for line in file]

msgs = [x.split('\t')[1].replace('\n', '') for x in lines]
emdbngs = [embeddings([x]) for x in msgs]
indx = list(range(1, len(msgs) + 1))

data_to_insert = [indx, msgs, emdbngs]
print(data_to_insert)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
# connect to the milvus DB

connections.connect (
    alias="default",
    host="localhost",
    port="19530"
)

In [10]:
# Field schema

id = FieldSchema (
    name="id",
    dtype=DataType.INT64,
    is_primary=True
)
message = FieldSchema (
    name="message",
    dtype=DataType.VARCHAR,
    max_length=6000
)
message_vec = FieldSchema (
    name="message_embeddings",
    dtype=DataType.FLOAT_VECTOR,
    dim=512
)

# collection schema 
collection_schema = CollectionSchema (
    fields = [id, message, message_vec], 
    description = "Spam SMS collection"
)

# create the collection
collection = Collection(
    name = "Spam_test",
    schema = collection_schema,
    using = 'default'
)
utility.list_collections()

['Spam_test', 'Album1']

In [11]:
# insert the data into the collection
data_insert = collection.insert(data_to_insert)

In [12]:
# create a index 
index_params = {
    "metric_type": "L2",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 1024},
    "index_name": "SMS_IVF_FLAT_TEST"
}

# index on vector field
collection.create_index(
    field_name="message_embeddings",
    index_params=index_params
)

Status(code=0, message=)

In [13]:
# load the collection 

collection.load(replica_number=1)

In [18]:
# test message 
test_message = ['prize']
test_message_vector = embeddings(test_message)

In [19]:
# vector similarity search 

search_params = {'metric_type': 'L2', 'params': {'nprobe': 64}}

results = collection.search(
    data = [test_message_vector],
    anns_field = 'message_embeddings',
    param = search_params,
    limit = 10,
    expr = None,
    output_fields = ['message']
)

for result in results[0]:
    print(result)

id: 3377, distance: 1.1559594869613647, entity: {'message': ':) '}
id: 4825, distance: 1.1559594869613647, entity: {'message': ':-) :-)'}
id: 4048, distance: 1.235376238822937, entity: {'message': 'Win a £1000 cash prize or a prize worth £5000'}
id: 931, distance: 1.2658859491348267, entity: {'message': 'money!!! you r a lucky winner ! 2 claim your prize text money 2 88600 over £1million to give away ! ppt150x3+normal text rate box403 w1t1jy'}
id: 2221, distance: 1.2849786281585693, entity: {'message': 'You have WON a guaranteed £1000 cash or a £2000 prize. To claim yr prize call our customer service representative on 08714712379 between 10am-7pm Cost 10p'}
id: 5013, distance: 1.2872653007507324, entity: {'message': 'You have WON a guaranteed £1000 cash or a £2000 prize. To claim yr prize call our customer service representative on 08714712412 between 10am-7pm Cost 10p'}
id: 277, distance: 1.3115980625152588, entity: {'message': 'Thanx...'}
id: 720, distance: 1.3198232650756836, entity

In [20]:
collection.release()